In [1]:
!pip install web3 pandas requests scikit-learn tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.5 MB/s eta 0:00:00


# New section

In [3]:
wallets_df = pd.read_csv("/content/Wallet id - Sheet1.csv")
wallets_df.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [12]:
import requests

address = "0xfaa0768bde629806739c3a4620656c5d26f44ef2"  # test wallet

url = "https://api.etherscan.io/api"
params = {
    "module": "account",
    "action": "txlist",
    "address": address,
    "startblock": 0,
    "endblock": 99999999,
    "sort": "asc",
    "apikey": "IM9KKSNMI7CN8EEPRZE7DWDRJ5R5Q9QZQ5"
}

response = requests.get(url, params=params)
data = response.json()

print("Status:", data["status"])
print("Message:", data["message"])
print("Number of transactions returned:", len(data["result"]))


Status: 1
Message: OK
Number of transactions returned: 526


In [13]:
import pandas as pd

# Load your CSV (ensure it's uploaded in Colab and the filename matches)
wallets_df = pd.read_csv("/content/Wallet id - Sheet1.csv")

# Preview
wallets_df.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [14]:
import requests
import time

API_KEY = "IM9KKSNMI7CN8EEPRZE7DWDRJ5R5Q9QZQ5"

def fetch_transactions(address):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] == "1":
        return data["result"]
    else:
        return []


In [16]:
import pandas as pd
import requests
import time
from tqdm import tqdm

wallets_df = pd.read_csv("/content/Wallet id - Sheet1.csv")
wallets = wallets_df["wallet_id"].tolist()

ETHERSCAN_API_KEY = "IM9KKSNMI7CN8EEPRZE7DWDRJ5R5Q9QZQ5"

compound_addresses = {
    "Comptroller": "0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b",
    "cETH": "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    "cDAI": "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "cUSDC": "0x39aa39c021dfbae8fac545936693ac917d5e7563",
    "cUSDT": "0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9",
    "cWBTC": "0xccf4429db6322d5c611ee964527d42e5d685dd6a",
    "cUNI": "0x35a18000230da775cac24873d00ff85bccded550",
    "cLINK": "0xface851a4921ce59e912d19329929ce6da6eb0c7",
}
compound_contracts = set(addr.lower() for addr in compound_addresses.values())

def fetch_transactions(address):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] != "1":
        return []
    return data["result"]

wallet_compound_data = {}

for wallet in tqdm(wallets):
    try:
        txs = fetch_transactions(wallet)
        compound_txs = [tx for tx in txs if tx["to"] and tx["to"].lower() in compound_contracts]
        wallet_compound_data[wallet] = compound_txs
        time.sleep(0.25)  # ✅ 0.25s delay to avoid Etherscan rate limits (~4 calls/sec)
    except Exception as e:
        print(f"Error fetching data for {wallet}: {e}")
        wallet_compound_data[wallet] = []

sample_wallet = wallets[0]
print(f"\nSample Wallet: {sample_wallet}")
print(f"Compound Tx Count: {len(wallet_compound_data[sample_wallet])}")
print("First Compound TX (if any):")
if wallet_compound_data[sample_wallet]:
    print(wallet_compound_data[sample_wallet][0])
else:
    print("No Compound activity.")


100%|██████████| 103/103 [01:01<00:00,  1.69it/s]


Sample Wallet: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Compound Tx Count: 25
First Compound TX (if any):
{'blockNumber': '8472545', 'blockHash': '0x671c321d574442aa6d5201f10248038051b0fcadf74438105e53b6dea20b5d40', 'timeStamp': '1567451347', 'hash': '0x1c61798644a09960097e8d16570248b3e899e5437ca3ef15febcf67a10a4017d', 'nonce': '696', 'transactionIndex': '24', 'from': '0x0039f22efb07a647557c7c5d17854cfd6d489ef3', 'to': '0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b', 'value': '0', 'gas': '450000', 'gasPrice': '2100000000', 'input': '0xc2998238000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000070000000000000000000000006c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e000000000000000000000000f5dce57282a584d2746faf1593d3121fcac444dc0000000000000000000000004ddc2d193948926d02f9b1fe9e1daa0718270ed5000000000000000000000000158079ee67fce2f58472a96584a73c7ab9ac95c100000000000000000000000039aa39c021dfbae8fac545936693ac917d5e7563000

In [17]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

HIGH_RISK_METHODS = ["repayBorrow", "borrow"]
LOW_RISK_METHODS = ["mint", "redeem", "supply", "enterMarkets"]

risk_features = []

for wallet, txs in wallet_compound_data.items():
    total_txs = len(txs)
    borrow_txs = sum(1 for tx in txs if "borrow" in tx.get("functionName", "").lower())
    repay_txs = sum(1 for tx in txs if "repay" in tx.get("functionName", "").lower())
    supply_txs = sum(1 for tx in txs if "mint" in tx.get("functionName", "").lower())
    redeem_txs = sum(1 for tx in txs if "redeem" in tx.get("functionName", "").lower())

    unique_methods = len(set(tx.get("functionName") for tx in txs if tx.get("functionName")))

    if total_txs == 0:
        borrow_ratio = 1
        supply_ratio = 0
    else:
        borrow_ratio = borrow_txs / total_txs
        supply_ratio = supply_txs / total_txs

    risk_features.append({
        "wallet": wallet,
        "total_txs": total_txs,
        "borrow_ratio": borrow_ratio,
        "supply_ratio": supply_ratio,
        "unique_methods": unique_methods,
        "borrow_count": borrow_txs,
        "repay_count": repay_txs
    })


features_df = pd.DataFrame(risk_features)


features_df["raw_risk"] = (
    features_df["borrow_ratio"] * 0.5
    - features_df["supply_ratio"] * 0.3
    - features_df["repay_count"] * 0.05
    - features_df["unique_methods"] * 0.05
)


scaler = MinMaxScaler(feature_range=(0, 1000))
features_df["score"] = scaler.fit_transform(-features_df["raw_risk"].values.reshape(-1, 1)).flatten().round(0).astype(int)


score_df = features_df[["wallet", "score"]]
score_df.columns = ["wallet_id", "score"]
score_df.to_csv("wallet_risk_scores.csv", index=False)


score_df.head(10)


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,885
1,0x06b51c6882b27cb05e712185531c1f74996dd988,680
2,0x0795732aacc448030ef374374eaae57d2965c16c,680
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,680
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,680
5,0x104ae61d8d487ad689969a17807ddc338b445416,680
6,0x111c7208a7e2af345d36b6d4aace8740d61a3078,600
7,0x124853fecb522c57d9bd5c21231058696ca6d596,680
8,0x13b1c8b0e696aff8b4fee742119b549b605f3cbc,680
9,0x1656f1886c5ab634ac19568cd571bc72f385fdf7,0
